### **DEPENDENCIES**

In [1]:
%load_ext autoreload

In [2]:
import sys
import time
import guidance
sys.path.append("..")
%autoreload 2
import pathway as pw
import team_ninety_one as tno

from guidance import models, gen, user, assistant, system, select

### **ENV & CONFIG**

In [6]:
openai_settings = tno.settings.OpenAISettings()

### **MODEL**

In [7]:
lm = models.OpenAI(
    model="gpt-4o-mini",
    api_key=openai_settings.api_key,
    echo=True,
)

### **MCTSwRAG**

In [ ]:
x = """\
ROOT: {root}

Can you resolve ROOT with the chain of thought until now? Say YES
only if you are very sure, otherwise we can think more. Tell me in 
one word, YES or NO.
"""


def mcts_action(fn):
    @guidance(dedent=False)
    def wrapper(lm, *args, **kwargs):
        action_descr = f"{fn.__name__}|"
        for i, arg in enumerate(args):
            action_descr += f"arg{i + 1}: {arg}|"
        for key in sorted(kwargs.keys()):
            action_descr += f"{key}: {kwargs[key]}|"
        lm = lm.set("action", action_descr)
        lm = fn(lm, *args, **kwargs)
        lm_temp = lm.copy()
        with user():
            lm_temp += f"""
            ROOT: {lm_temp["ROOT"]}
            
            Can you resolve ROOT with the chain of thought until now? Say YES
            only if you are very sure, otherwise we can think more. Tell me in 
            one word, YES or NO.
            """
        with assistant():
            lm_temp += select(["YES", "NO"], "is_terminal")
        lm = lm.set("is_terminal", lm_temp["is_terminal"])
        return lm

    return wrapper


@mcts_action
def progg(lm):
    with user():
        lm += f"The mass of the car is 4 kg"
        lm += f"The acceleration of the car is 5 m/s^2"
    return lm


lm = lm.set("ROOT", "What is the force with which car strikes?")
lm += progg()

In [53]:
lform = f"""
LEGAL AGREEMENT FORM
--------------------
This agreement is entered into by and between:
Name    :
Age     : {gen()}
Address : {gen()}
Phone   : {gen()}
Email   : {gen()}

Terms and conditions apply as per the governing law. The above information is 
accurate to the best of our knowledge.
"""


class Person(pw.Schema):
    name: str
    age: int
    address: str
    phone: str
    email: str


people = pw.io.csv.read(
    "/home/meghnaad/workspace-b/interiit-pathway-rag/data/people.csv",
    schema=Person,
    mode="static",
)

fil = people.filter(people.name == "Alice Johnson")

# convert row to a string
pdf = pw.debug.table_to_pandas(fil)
str(pdf)

[2024-12-12T02:54:34]:INFO:Preparing Pathway computation
[2024-12-12T02:54:34]:INFO:PosixLikeReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-12-12T02:54:34]:INFO:PosixLikeReader-0: 3 entries (2 minibatch(es)) have been sent to the engine
[2024-12-12T02:54:34]:WARNING:PosixLikeReader-0: Closing the data source


'                                      name  age      address     phone  \\\n^50PYJ3F9JQT5G84ZFMCQYFD8GM  Alice Johnson   30  123 Main St  555-1234   \n\n                                           email  \n^50PYJ3F9JQT5G84ZFMCQYFD8GM  alice.j@example.com  '

In [55]:
# Store the raw string
raw_lform = r"""
LEGAL AGREEMENT FORM
--------------------
This agreement is entered into by and between:
Name    :
Age     : {gen()}
Address : {gen()}
Phone   : {gen()}
Email   : {gen()}

Terms and conditions apply as per the governing law. The above information is 
accurate to the best of our knowledge.
"""

# Function to convert raw string to f-string
def convert_to_fstring(raw_str):
    return eval(f"f'''{raw_str}'''")

# Example usage
lform = convert_to_fstring(raw_lform)
print(raw_lform)
print(lform)


LEGAL AGREEMENT FORM
--------------------
This agreement is entered into by and between:
Name    :
Age     : {gen()}
Address : {gen()}
Phone   : {gen()}
Email   : {gen()}

Terms and conditions apply as per the governing law. The above information is 
accurate to the best of our knowledge.


LEGAL AGREEMENT FORM
--------------------
This agreement is entered into by and between:
Name    :
Age     : {{G|139639515199568|G}}
Address : {{G|139639515198000|G}}
Phone   : {{G|139639515191056|G}}
Email   : {{G|139639516726336|G}}

Terms and conditions apply as per the governing law. The above information is 
accurate to the best of our knowledge.



In [58]:
import copy

# Assuming you have a nested object
nested_object = {
    'key1': [1, 2, 3],
    'key2': {'subkey1': 'value1', 'subkey2': 'value2'}
}

# Create a deep copy of the nested object
deep_copied_object = copy.deepcopy(nested_object)

# Now, deep_copied_object is a deep copy of nested_object
# test
nested_object['key1'].append(4)
print(deep_copied_object)
print(nested_object)

{'key1': [1, 2, 3], 'key2': {'subkey1': 'value1', 'subkey2': 'value2'}}
{'key1': [1, 2, 3, 4], 'key2': {'subkey1': 'value1', 'subkey2': 'value2'}}


In [64]:
x = """
Legal Question: "Can pre-nups be made legal in India?"

MCTS Log:
Iteration 1:
    Selection:
        Selected Node: Root
    Expansion:
        Expanded Node: Root
        Added Nodes: [Node1, Node2]
    Simulation:
        Simulating Node: Node1
        Simulation Reward: 0.2
        Simulating Node: Node2
        Simulation Reward: 0.3
    Update:
        Updated Node: Root with Reward: 0.25

Iteration 2:
    Selection:
        Selected Node: Node2
    Expansion:
        Expanded Node: Node2
        Added Nodes: [Node3, Node4]
    Simulation:
        Simulating Node: Node3
        Simulation Reward: 0.4
        Simulating Node: Node4
        Simulation Reward: 0.1
    Update:
        Updated Node: Node2 with Reward: 0.25

Iteration 3:
    Selection:
        Selected Node: Node3
    Expansion:
        Expanded Node: Node3
        Added Nodes: [Node5, Node6]
    Simulation:
        Simulating Node: Node5
        Simulation Reward: 0.5
        Simulating Node: Node6
        Simulation Reward: 0.2
    Update:
        Updated Node: Node3 with Reward: 0.35

Iteration 4:
    Selection:
        Selected Node: Node5
    Expansion:
        Expanded Node: Node5
        Added Nodes: [Node7, Node8]
    Simulation:
        Simulating Node: Node7
        Simulation Reward: 0.6
        Simulating Node: Node8
        Simulation Reward: 0.3
    Update:
        Updated Node: Node5 with Reward: 0.45

Iteration 5:
    Selection:
        Selected Node: Node7
    Expansion:
        Expanded Node: Node7
        Added Nodes: [Node9, Node10]
    Simulation:
        Simulating Node: Node9
        Simulation Reward: 0.7
        Simulating Node: Node10
        Simulation Reward: 0.4
    Update:
        Updated Node: Node7 with Reward: 0.55

Final Result:
    Best Path: Root -> Node2 -> Node3 -> Node5 -> Node7 -> Node9
    Total Reward: 0.55
"""

# pretty print
from pprint import pprint

pprint(x)
# final answer

('\n'
 'Legal Question: "Can pre-nups be made legal in India?"\n'
 '\n'
 'MCTS Log:\n'
 'Iteration 1:\n'
 '    Selection:\n'
 '        Selected Node: Root\n'
 '    Expansion:\n'
 '        Expanded Node: Root\n'
 '        Added Nodes: [Node1, Node2]\n'
 '    Simulation:\n'
 '        Simulating Node: Node1\n'
 '        Simulation Reward: 0.2\n'
 '        Simulating Node: Node2\n'
 '        Simulation Reward: 0.3\n'
 '    Update:\n'
 '        Updated Node: Root with Reward: 0.25\n'
 '\n'
 'Iteration 2:\n'
 '    Selection:\n'
 '        Selected Node: Node2\n'
 '    Expansion:\n'
 '        Expanded Node: Node2\n'
 '        Added Nodes: [Node3, Node4]\n'
 '    Simulation:\n'
 '        Simulating Node: Node3\n'
 '        Simulation Reward: 0.4\n'
 '        Simulating Node: Node4\n'
 '        Simulation Reward: 0.1\n'
 '    Update:\n'
 '        Updated Node: Node2 with Reward: 0.25\n'
 '\n'
 'Iteration 3:\n'
 '    Selection:\n'
 '        Selected Node: Node3\n'
 '    Expansion:\n'
 '        Ex